### Model deployment as a batch - scoring service.

Here we use hopsworks for our feature store. In this notebook we backfill our feature store with data startin from 2022.

In [52]:
import sys 
sys.path.append(r"C:\Users\User\capstone_project")

In [53]:
HOPSWORKS_PROJECT_NAME = 'predicting_taxi_demand'

In [54]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [55]:
import os # to access environment variables
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

Download historical data from the website starting 2022 todate.

In [56]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage
File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
File 2023-06 was already in local storage
File 2023-07 was already in local storage
File 2023-08 was already in local storage
File 2023-09 was already in local storage
File 2023-10 was already in local storage
2023-11 file is not available
2023-12 file is not available
2024-01 file is 

Check how many rows of data.

In [57]:
print(f'{len(rides)=:,}')

len(rides)=71,247,922


We have over 71 million rows.

Now, let us transform the data into time series data.

In [58]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:04<00:00, 54.33it/s]


In [80]:

# string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype('int64') // 10**6



In [81]:
%pip install hopsworks==3.4.*

Note: you may need to restart the kernel to use updated packages.


In [82]:
import hopsworks

In [83]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.4) by running 'pip install hopsworks==3.4.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/369813


Now we have connected to the project.

Next let us connect to the feature store.

In [84]:
%pip install hsfs[python] 



Note: you may need to restart the kernel to use updated packages.


In [85]:
import hsfs
conn = hsfs.connection()
feature_store = conn.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [46]:
# feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


Let us set the feature group

In [86]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

Let us create the feature group.

In [87]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour'
)

Inserting data into feature group.

In [89]:
"""
feature_group.insert(ts_data, write_options={"wait_for_job": False})
"""

'\nfeature_group.insert(ts_data, write_options={"wait_for_job": False})\n'